<a href="https://colab.research.google.com/github/won-seop/CAPSTONE-DESIGN/blob/main/data_collector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-api-python-client

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

In [ ]:

DEVELOPER_KEY = "" # 유튜브 API 키값 입력
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

In [ ]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)
search_response = youtube.search().list(
    q = "트라메 - TRAME", # 검색 채널  (이것만 바꿔가면서 하면됨)
    order = 'relevance', # 검색 결과 유형
    part = 'snippet',
    maxResults = 1 # 최대 호출 개수, 1로 설정한 이유는 채널ID 만 가져오면 되므로
).execute()

In [ ]:
# 재생목록 가져오기
channel_id = search_response['items'][0]['snippet']['channelId']

playlists = youtube.playlists().list(
    channelId = channel_id,
    part = 'snippet',
    maxResults = 25 # 채널의 플레이리스트 다 가져오도록
).execute()

import pandas as pd

df = pd.DataFrame()
for item in playlists['items']:
  item_dict = {}
  item_dict['playlist'] = item['snippet']['title']
  item_dict['playlist_id'] = item['id']
  df = df.append(item_dict, ignore_index = True)

<ipython-input-84-4801f268a59e>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(item_dict, ignore_index = True)
<ipython-input-84-4801f268a59e>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(item_dict, ignore_index = True)
<ipython-input-84-4801f268a59e>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(item_dict, ignore_index = True)
<ipython-input-84-4801f268a59e>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(item_dict, ignore_index = True)
<ipython-input-84-4801f268a59e>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

In [ ]:
# 재생목록 내 동영상 가져오기
video_titles = []
video_ids = []

for id in df['playlist_id']:
  playlistitems_list_request = youtube.playlistItems().list(
      playlistId = id,  # 재생목록 id 입력
      part = 'snippet',
  )

  while playlistitems_list_request:
    playlistitems_list_response = playlistitems_list_request.execute()

    for item in playlistitems_list_response['items']:
      title = item['snippet']['title']
      video_id = item['snippet']['resourceId']['videoId']

      video_titles.append(title)
      video_ids.append(video_id)

    playlistitems_list_request = youtube.playlistItems().list_next(
        playlistitems_list_request, playlistitems_list_response
    )

video_df = pd.DataFrame()
video_df['video_title'] = video_titles
video_df['video_id'] = video_ids

video_df.drop_duplicates(inplace=True, ignore_index=True) # 재생목록에서 중복된 영상 제거

if len(video_df) < 500:
  video_df = video_df
else:
  video_df = video_df.sample(n=500, ignore_index=True) # 데이터프레임에서 무작위로 500개 추출

video_df

,video_title,video_id
0,당신은 전자기기 없이 얼마나 버틸 수 있습니까? ⚡️ #테크룸 티저 ⚡️ #딕헌터 ...,JaKXNIkaDe4
1,"테크룸에서 살아남는 방법 ⚡️ #테크룸 티저.2 ⚡️(조재원, JM, 이태준, 뽀구...",QvgWavEIFAM
2,첫 판부터 장난질이냐? ⚡️ #테크룸 EP.1 ⚡️테크 서바이벌 #뽀구미 #JM #...,H6vwVEKshts
3,보물찾기 미션이 시작됩니다 ⚡️ #테크룸 EP.2 ⚡️테크 서바이벌 #JM #이태준...,f0AQSd0w9Pk
4,갤럭시 vs 아이폰 - 소수결 게임 ⚡️ #테크룸 EP.3 ⚡️테크 서바이벌 #이태...,0_CP7th0ZUM
...,...,...
429,[라운드원] 소믈리에가 직접 알려주는 와인 꿀팁🍷,189Z_NnPusw
430,Private video,rOsLJhzwQZk
431,[라운드원] 음식 감수하시겠습니까? : 지옥 간접경험 MC 해나 넉다운,NKQXt_9crYI
432,[라운드원] 소비요괴 김해나의 진실 : 비긴즈,OGPhqObFIBw


In [ ]:
# 제목, 조회수 모은 데이터프레임 만들기
import numpy as np
import random

titles = [] # 동영상 제목
views = [] # 조회수

for i in range(len(video_df)):
  request = youtube.videos().list(
      id = video_df['video_id'][i], # 동영상 id 입력
      part = 'snippet, contentDetails, statistics'
  )
  response = request.execute()

  if response['items'] == []: # 동영상 정보가 없을 경우 '-'로 입력
    titles.append(np.nan)
    views.append(np.nan)

  else:
    titles.append(response['items'][0]['snippet']['title'])
    views.append(response['items'][0]['statistics'].get('viewCount'))


detail_df = pd.DataFrame([titles, views]).T
detail_df.columns = ['title', 'view']

detail_df.dropna(axis=0, inplace=True)

detail_df

,title,view
0,당신은 전자기기 없이 얼마나 버틸 수 있습니까? ⚡️ #테크룸 티저 ⚡️ #딕헌터 ...,107232
1,"테크룸에서 살아남는 방법 ⚡️ #테크룸 티저.2 ⚡️(조재원, JM, 이태준, 뽀구...",159068
2,첫 판부터 장난질이냐? ⚡️ #테크룸 EP.1 ⚡️테크 서바이벌 #뽀구미 #JM #...,284428
3,보물찾기 미션이 시작됩니다 ⚡️ #테크룸 EP.2 ⚡️테크 서바이벌 #JM #이태준...,242538
4,갤럭시 vs 아이폰 - 소수결 게임 ⚡️ #테크룸 EP.3 ⚡️테크 서바이벌 #이태...,308725
...,...,...
428,[라운드원] 이런것도 있어? 삐에로쑈핑 BEST 5,50119
429,[라운드원] 소믈리에가 직접 알려주는 와인 꿀팁🍷,54066
431,[라운드원] 음식 감수하시겠습니까? : 지옥 간접경험 MC 해나 넉다운,76718
432,[라운드원] 소비요괴 김해나의 진실 : 비긴즈,41173


In [ ]:
# 본인 Drive에 저장해서 관리
import os
path = '/content/drive/My Drive/data2.csv'
with open(path, 'a', encoding = 'utf-8-sig') as f:
  detail_df.to_csv(f, header=False, index=False)